In [1]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# 変換したい画像へのパス
target_path = './'

# スタイル画像へのパス
style_reference_image_path = './'

width, height = load_img(target_path).size
img_height = 400
img_width = int(width * img_height / height)

IsADirectoryError: [Errno 21] Is a directory: './'

In [3]:
# 補助関数
import numpy as np 
from tensorflow.keras.applications import vgg19

def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

def deprocess_image(x):
    # ImageNetから平均ピクセル値を取り除くことにより，中心を０に設定
    # これにより，vgg19.preprocess_input二よって実行される変換が逆になる
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.799
    x[:, :, 2] += 123.68

    # 画像をBGRからRGBに変換
    # これもvgg19.preprocess_inputの変換を逆にするために措置
    x = x[:, :, ::-1]

    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [ ]:
# 学習済みのvgg19ネットワークを読み込み，3つの画像に適用
from tensorflow.keras import backend as K 

target_image = K.constant(preprocess_image(target_path))
style_reference_image = K.constant(preprocess_image(style_reference_image_path))

# 生成された画像を保持するプレースホルダー
combination_image = K.placeholder((1, img_height, img_width))

# ３つの画像を１つのバッチにまとめる
input_tensor = K.conatenate([target_image, style_reference_image, combination_image], axis=0)

# ３つの画像からなるバッチを入力として使用するvgg19モデルを構築
# このモデルは学習済みのImageNetの重みが読み込まれる
model = vgg19(input_tensor=input_tensor,
              weights='imagenet',
              include_top=False)
print('Model loaded.')

In [ ]:
# コンテンツの損失関数
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

In [ ]:
# スタイルの損失関数
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S - C)) / (4 * (channels ** 2) * (size ** 2))

In [ ]:
# 全変動損失関数
def total_variation_loss(x):
    a = K.square(
        x[:, :img_height - 1, :img_width - 1, :] - x[:, 1:, :img_width - 1, :])
    b = K.square(
        x[:, :img_height - 1, :img_width - 1, :] - x[:, :img_height - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

In [ ]:
# 最小かの対象となる最終的な損失関数を定義
# 層の名前を活性化テンソルにマッピングするディクショナリ
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
# コンテンツの損失関数に使用する層の名前
content_layer = 'block5_conv2'
# スタイルの損失関数に使用する層の名前
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']
# 損失関数の加重平均の重み
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025

# 全てのコンポーネントをこのスカラー変数に追加することで，損失関数を定義する
loss = K.variable(0.)
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss += content_weight * content_loss(target_image_features,
                                      combination_features)
for layer_name in style_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(style_layers)) * sl
# 全変動損失関数を追加
loss += total_variation_weight * total_variation_loss(combination_image)

In [ ]:
# 勾配降下法のプロセスを定義
# 損失関数を元に，生成された画像の勾配を取得
grads = K.gradients(loss, combination_image)[0]

# 現在の損失関数の値と勾配の値を取得する関数
fetch_loss_and_grads = K.function([combination_image], [loss, grads])

# このクラスは損失関数の値と勾配の値を２つのメソッド呼び出しを通じて取得で
# きるようにfetch_loss_and_gradsをラッピングする．この２つのメソッド呼び出しは，
# ここで使用するScipyのオプティマイザによって要求される
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator()
